<h3>Задание 7. Транспортная задача</h3><br>
Дана транспортная задача: A<sub>1</sub>, A<sub>2</sub>, A<sub>3</sub> – поставщики с запасами a<sub>1</sub>, a<sub>2</sub>, a<sub>3</sub> однородного груза,<br>
B<sub>1</sub>, B<sub>2</sub>, B<sub>3</sub>, B<sub>4</sub>, B<sub>5</sub> – потребители с потребностями b<sub>1</sub>, b<sub>2</sub>, b<sub>3</sub>, b<sub>4</sub>, b<sub>5</sub>.<br>
Матрица тарифов C = (c<sub>ij</sub>), где i = 1, 2, 3 и j = 1, 2, 3, 4, 5, содержит стоимости перевозки единицы груза
из пункта A<sub>i</sub> в пункт B<sub>j</sub>.<br>
Требуется найти минимальный по стоимости план перевозки груза от поставщиков
к потребителям такой, чтобы был вывезен весь груз и все потребности были удовлетворены.<br>
1. Убедитесь, что транспортная задача закрытого типа.<br>
2. Найдите какой-либо допустимый план перевозки груза (можно использовать методы северо-западного угла, минимальной стоимости или Фогеля).<br>
3. Найдите оптимальный план перевозки груза (план минимальной стоимости) методом потенциалов.<br>
4. Проверьте результаты вычислений на компьютере.

In [53]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
np.set_printoptions(suppress=True)

In [54]:
C = np.array([list(map(int, input(f'Введите {i+1} строку матрицы C (числа через пробел):\n').split())) for i in range(3)])
print('\nМатрица C:')
print(C)

Введите 1 строку матрицы C (числа через пробел):
16 7 13 10 22
Введите 2 строку матрицы C (числа через пробел):
6 15 12 9 4
Введите 3 строку матрицы C (числа через пробел):
14 11 5 17 8

Матрица C:
[[16  7 13 10 22]
 [ 6 15 12  9  4]
 [14 11  5 17  8]]


In [55]:
a1, a2, a3 = map(int, input('Введите через пробел числа a1, a2, a3:\n').split())

Введите через пробел числа a1, a2, a3:
280 370 240


In [56]:
A = np.array([a1, a2, a3])

In [57]:
b1, b2, b3, b4, b5 = map(int, input('Введите через пробел числа b1, b2, b3, b4, b5:\n').split())

Введите через пробел числа b1, b2, b3, b4, b5:
150 120 210 260 150


In [58]:
B = np.array([b1, b2, b3, b4, b5])

In [59]:
N = len(A)
M = len(B)

In [60]:
df = pd.DataFrame(C, columns=B, index=A)
df.columns.name = 'aᵢ \\ bⱼ'
df

aᵢ \ bⱼ,150,120,210,260,150
280,16,7,13,10,22
370,6,15,12,9,4
240,14,11,5,17,8


In [61]:
if sum(A) == sum(B):
    print('Задача закрытого типа')
else:
    print('Задача открытого типа')

Задача закрытого типа


Допустимый план перевозки груза методом минимальной стоимости

In [101]:
P = np.zeros_like(C)
INF = 1_000_000_007
C_ = C.copy()
A_ = A.copy()
B_ = B.copy()
i = 0
j = 0
while not np.all(C_ == INF):
    i, j = np.unravel_index(C_.argmin(), C_.shape)
    P[i][j] = min(A_[i], B_[j])
    if A_[i] <= B_[j]:
        C_[i].fill(INF)
        B_[j] -= A_[i]
        A_[i] = 0
    else:
        C_[:, j].fill(INF)
        A_[i] -= B_[j]
        B_[j] = 0    
if np.count_nonzero(P) == N + M - 1:
    print('Опорный план является невырожденным')
else:
    print('Опорный план является вырожденным')

F = 0
for i in range(N):
    for j in range(M):
        F += C[i][j] * P[i][j]
print('Полученный план перевозки груза:\n', P)
print('Значение целевой функции для этого опорного плана F =', F)

Опорный план является невырожденным
Полученный план перевозки груза:
 [[  0 120   0 160   0]
 [150   0   0  70 150]
 [  0   0 210  30   0]]
Значение целевой функции для этого опорного плана F = 6130


Оптимальный план перевозки груза

In [63]:
#A_ub - матрица коэф. при x1,x2,...,xn в ограничениях, A_ub @ x <= b_ub (E @ x <= A)
#c0 - коэффициенты перед переменными в целевой функции
A_ub = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
b_ub = A

#A_ub - матрица коэф. при x1,x2,...,xn в ограничениях, A_eq @ x = b_eq (E @ x = B)
#c0 - коэффициенты перед переменными в целевой функции
b_eq = B
A_eq = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])

res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})

X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 169.0               
0.1202059322604     0.1202059322604     0.1202059322603     0.8901452841057  0.1202059322604     1147.263137699      
0.03997616213494    0.03997616213494    0.03997616213489    0.6858931492159  0.03997616213493    2081.479451867      
0.01285243958581    0.01285243958581    0.01285243958579    0.6841818430431  0.0128524395858     3484.251347368      
0.003264438710608   0.003264438710608   0.003264438710605   0.7766349401104  0.003264438710609   5111.196703651      
0.000209819860793   0.0002098198607929  0.000209819860792   0.9746110433056  0.000209819860794   5940.84892623       
3.095662218527e-08  3.095662222195e-08  3.095662154853e-08  0.9998527379678  3.095662419011e-08  6009.989746303      
1.602107148119e-12  1.602105100264e-12  1.602482852297e-1

C:\Users\davem\AppData\Local\Temp/ipykernel_30772/3296677646.py:11: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})
C:\Users\davem\AppData\Local\Temp/ipykernel_30772/3296677646.py:11: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})


In [64]:
print('Оптимальный план перевозки груза X*:\n', X)
print('Оптимальная стоимость перевозки:\nF_min = F(X*) =', F_min)

Оптимальный план перевозки груза X*:
 [[  0. 120.   0. 160.   0.]
 [150.   0.   0. 100. 120.]
 [  0.   0. 210.   0.  30.]]
Оптимальная стоимость перевозки:
F_min = F(X*) = 6010.0
